### Import Required Modules

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import pickle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
# PATH_TO_FOLDER = "drive/MyDrive/Name2Demographics/"
PATH_TO_FOLDER = "../../../"
%cd $PATH_TO_FOLDER

/home/souvic/mounted/btp/vahini/Name2Demographics


In [3]:
import sys  
sys.path.insert(0, 'Models/ERData/PreProcessing/')

from er_preprocess import *

### Train Model on Data

In [4]:
er_df = ERData_MajorityLabel()

In [5]:
train, test = train_test_split(er_df, test_size=0.3)

In [6]:
print(train.shape, ":", test.shape)

(5004642, 2) : (2144847, 2)


In [ ]:
# train, val, test = splitTrainTestVal(er_df[:500000], 0.8 ,0, 'Name')

In [ ]:
# train1, _, test1 = splitTrainTestVal(er_df[100000:500000], 0.8 ,0, 'Name')

In [16]:
sgd_pipe = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 6), lowercase = True, stop_words = "english")),
    ('tfidf', TfidfTransformer(sublinear_tf = True)),
    ('clf', SGDClassifier(class_weight="balanced",loss="squared_loss",max_iter=10000, random_state=42, validation_fraction = 0.3, average=1000
                          , learning_rate='adaptive', eta0=0.003))
])

In [17]:
sgd_pipe.fit(train['Name'].values.astype('str'), train['Gender'].values.astype('str'))

Pipeline(steps=[('vect',
                 CountVectorizer(ngram_range=(1, 6), stop_words='english')),
                ('tfidf', TfidfTransformer(sublinear_tf=True)),
                ('clf',
                 SGDClassifier(average=1000, class_weight='balanced',
                               eta0=0.003, learning_rate='adaptive',
                               loss='squared_loss', max_iter=10000,
                               random_state=42, validation_fraction=0.3))])

In [18]:
y_pred=sgd_pipe.predict(test['Name'].values.astype('str'))
# print(classification_report(test['Gender'], y_pred, target_names = ['Male', 'Female']))

In [19]:
y_pred = y_pred.astype(str)

In [20]:
print(classification_report(test['Gender'].values.astype('str'), y_pred, digits=4))

              precision    recall  f1-score   support

         0.0     0.6574    0.9056    0.7618   1000777
         1.0     0.8768    0.5872    0.7033   1144070

    accuracy                         0.7358   2144847
   macro avg     0.7671    0.7464    0.7326   2144847
weighted avg     0.7744    0.7358    0.7306   2144847



### Save Model

In [21]:
filename = "Models/ERData/SGDClassifier/SavedModel/ERModel.pkl"
pickle.dump(sgd_pipe, open(filename, 'wb'))